In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import time 
from bs4 import BeautifulSoup
import csv
import pandas as pd
from selenium.webdriver.support.ui import Select
import tqdm

**1) Primera parte del Scrapper** 

Acá vamos a descargar los enlaces a cada proyecto de ley que es de nuestro interés

In [3]:
# Scrapper para descargar los enlaces a la página de seguimiento


driver = webdriver.Chrome()
driver.get("https://wb2server.congreso.gob.pe/spley-portal/#/expediente/search")


mis_proyectos = ['06805', '06791', '00005']
links = []


for i in mis_proyectos:
    input_field = driver.find_element("css selector", 'input[placeholder="Ingrese número"]')
    input_field.send_keys(i)
    input_field.send_keys(Keys.ENTER)
    time.sleep(10)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find("tbody", {"class": "p-datatable-tbody"})

    for row in table.find_all("tr"):
        link = row.a['href']
        links.append(link)

    input_field.clear()


In [3]:
df = pd.DataFrame(list(zip(mis_proyectos, links)), 
                    columns =['Número de Proyecto de Ley', 'Enlace'])
df = df.reset_index(drop=True)

df['Enlace'] = df['Enlace'].str.replace('#', 'https://wb2server.congreso.gob.pe/spley-portal/#')

df



,Número de Proyecto de Ley,Enlace
0,06805,https://wb2server.congreso.gob.pe/spley-portal...
1,06791,https://wb2server.congreso.gob.pe/spley-portal...
2,00005,https://wb2server.congreso.gob.pe/spley-portal...


In [4]:
link = df['Enlace'].iloc[2]    
link  

'https://wb2server.congreso.gob.pe/spley-portal/#/expediente/2021/5'

**2) Segunda parte del Scrapper**

Acá vamos a descargar la información que se actualiza en tiempo real de la página de proyectos de ley del Congreso de la República

In [5]:
driver = webdriver.Chrome()
driver.get("https://wb2server.congreso.gob.pe/spley-portal/#/expediente/2021/5618")


time.sleep(8)


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
table = soup.find("tbody", {"class": "p-datatable-tbody"})


l_fecha = []
l_estado = []
l_nley = []


for row in table.findAll('tr', class_='ng-star-inserted'):
    fecha = row.findAll('td')[0].text
    estado = row.findAll('td')[1].text
    nley = row.findAll('td')[3].text


    l_fecha.append(fecha)
    l_estado.append(estado)
    l_nley.append(nley)

In [6]:
df_1 = pd.DataFrame(list(zip(l_fecha, l_estado, l_nley)), 
                    columns =['Fecha del Proyecto', 'Estado', 'Descripción'])
df_1 = df_1.reset_index(drop=True)

# Esto limpia el texto de las observaciones 

df_1['Fecha del Proyecto'] = df_1['Fecha del Proyecto'].str.replace('Fecha', '')
df_1['Estado'] = df_1['Estado'].str.replace('Estado', '')
df_1['Descripción'] = df_1['Descripción'].str.replace('Detalle', '')

df_1

,Fecha del Proyecto,Estado,Descripción
0,05/11/2023,Publicada en el Diario Oficial El Peruano,FE DE ERRATAS
1,31/10/2023,Publicada en el Diario Oficial El Peruano,FE DE ERRATAS
2,27/10/2023,Publicada en el Diario Oficial El Peruano,LEY N° 31912 - LEY QUE APRUEBA CRÉDITOS SUPLEM...
3,26/10/2023,Promulgado/Presidente de la República,LEY N° 31912
4,24/10/2023,Autógrafa,SOBRE N° 63 - LEY QUE APRUEBA CRÉDITOS SUPLEME...
5,20/10/2023,APROBADO,\nASISTENCIA Y VOTACIÓN - TEXTO SUSTITUTORIO\n
6,20/10/2023,EN DEBATE - PLENO,TEXTO SUSTITUTORIO DE LA COMISIÓN DE PRESUPUES...
7,20/10/2023,EN DEBATE - PLENO,DURANTE LA SESIÓN DEL PLENO SE ACUMULÓ EL PROY...
8,20/10/2023,EN DEBATE - PLENO,DURANTE LA SESIÓN DEL PLENO SE DESACUMULÓ EL P...
9,20/10/2023,EN AGENDA DEL PLENO,"LA JUNTA DE PORTAVOCES, CON FECHA 20 DE OCTUBR..."


In [13]:
df_1.to_csv(r'D:\Proyectos\dc_proyectos\proyectos_de_ley\24_01_16_06805.csv')

**3) Comparador de tablas semanales**

Lo que queremos saber es si la tabla de la semana 1 ha sufrido alguna diferencia con respecto a la tabla 2

In [7]:
semana_1 = pd.read_csv(r'D:\Proyectos\dc_proyectos\proyectos_de_ley\12_01_16_06805.csv', index_col=0)

semana_2 = pd.read_csv(r'D:\Proyectos\dc_proyectos\proyectos_de_ley\24_01_16_06805.csv', index_col=0)

In [8]:
semana_1

,Fecha del Proyecto,Estado,Descripción
0,31/10/2023,Publicada en el Diario Oficial El Peruano,FE DE ERRATAS
1,27/10/2023,Publicada en el Diario Oficial El Peruano,LEY N° 31912 - LEY QUE APRUEBA CRÉDITOS SUPLEM...
2,26/10/2023,Promulgado/Presidente de la República,LEY N° 31912
3,24/10/2023,Autógrafa,SOBRE N° 63 - LEY QUE APRUEBA CRÉDITOS SUPLEME...
4,20/10/2023,APROBADO,\nASISTENCIA Y VOTACIÓN - TEXTO SUSTITUTORIO\n
5,20/10/2023,EN DEBATE - PLENO,TEXTO SUSTITUTORIO DE LA COMISIÓN DE PRESUPUES...
6,20/10/2023,EN DEBATE - PLENO,DURANTE LA SESIÓN DEL PLENO SE ACUMULÓ EL PROY...
7,20/10/2023,EN DEBATE - PLENO,DURANTE LA SESIÓN DEL PLENO SE DESACUMULÓ EL P...
8,20/10/2023,EN AGENDA DEL PLENO,"LA JUNTA DE PORTAVOCES, CON FECHA 20 DE OCTUBR..."
9,20/10/2023,Dictamen,EN MAYORÍA - FÓRMULA SUSTITUTORIA - LEY QUE AP...


In [9]:
semana_2

,Fecha del Proyecto,Estado,Descripción
0,05/11/2023,Publicada en el Diario Oficial El Peruano,FE DE ERRATAS
1,31/10/2023,Publicada en el Diario Oficial El Peruano,FE DE ERRATAS
2,27/10/2023,Publicada en el Diario Oficial El Peruano,LEY N° 31912 - LEY QUE APRUEBA CRÉDITOS SUPLEM...
3,26/10/2023,Promulgado/Presidente de la República,LEY N° 31912
4,24/10/2023,Autógrafa,SOBRE N° 63 - LEY QUE APRUEBA CRÉDITOS SUPLEME...
5,20/10/2023,APROBADO,\nASISTENCIA Y VOTACIÓN - TEXTO SUSTITUTORIO\n
6,20/10/2023,EN DEBATE - PLENO,TEXTO SUSTITUTORIO DE LA COMISIÓN DE PRESUPUES...
7,20/10/2023,EN DEBATE - PLENO,DURANTE LA SESIÓN DEL PLENO SE ACUMULÓ EL PROY...
8,20/10/2023,EN DEBATE - PLENO,DURANTE LA SESIÓN DEL PLENO SE DESACUMULÓ EL P...
9,20/10/2023,EN AGENDA DEL PLENO,"LA JUNTA DE PORTAVOCES, CON FECHA 20 DE OCTUBR..."


In [10]:
# Merge DataFrames A and B with indicator
merged_df = pd.merge(semana_2, semana_1, how='outer', indicator=True)

# Select rows where the indicator is only in 'left_only'
novedad = merged_df[merged_df['_merge'] == 'left_only']

novedad = novedad.drop(columns=['_merge'])

novedad

,Fecha del Proyecto,Estado,Descripción
0,05/11/2023,Publicada en el Diario Oficial El Peruano,FE DE ERRATAS
